In [3]:
%run visualisation.ipynb
%run read_graph.ipynb

import pixiedust

g = read_nba_full("../Scrapers/BasketballReferenceScraper/data/Players.csv",
                  "../Scrapers/BasketballReferenceScraper/data/Edges.csv",
                  True, True)

all11_20 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159, 1038, 1113, 1929, 3429, 4432, 1125, 863, 1588, 2586,4198,115,2925,1459,2235,1794,415,1460,564,4168,2384,1142,3134,1849,2145,1945,3283,3669,1446,718,1031,1898,2577,2492,3502,620,3079,3672,1995,3383,4609,4090,65,2930,1504]
nba11_20 = g.subgraph(all11_20)

all15_20 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159, 1038, 1113, 1929, 3429, 4432, 1125, 863, 1588, 2586,4198,115,2925,1459,2235,1794,415,1460,564,4168,2384,1142]
nba15_20 = g.subgraph(all15_20)

all19_20 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159]
nba19_20 = g.subgraph(all19_20)

Pixiedust database opened successfully


Reading full NBA graph, begin...
Reading file ../Scrapers/BasketballReferenceScraper/data/Players.csv, begin...
Data lines read: 4800
Reading file ../Scrapers/BasketballReferenceScraper/data/Edges.csv, begin...
Data lines read: 186607
Reading NBA vertices, begin...
Vertices read: 4800
Reading NBA edges, begin...
Weights loaded correctly
Edges read: 186607
Loading succesfull


In [4]:
def build_lpa(graph, output):
    result = graph.community_label_propagation()
    resultArray = result.membership
    i = 0
    for v in graph.vs:
        v["communities"] = [resultArray[i]]
        i += 1

    plot_full(graph, output, 1)

In [5]:
build_lpa(nba11_20, "all_stars_1120.svg")
build_lpa(nba15_20, "all_stars_1520.svg")
build_lpa(nba19_20, "all_stars_1920.svg")

Plotting graph with 83 vertices and 344 edges, begin...
Plotting sucessful
Plotting graph with 60 vertices and 143 edges, begin...
Plotting sucessful
Plotting graph with 39 vertices and 37 edges, begin...
Plotting sucessful


In [23]:
def my_basic_lpa(graph, output):
    counters = []
    vertices_count = len(graph.vs)
    
    i = 0
    for v in graph.vs:
        v["communities"] = [i]
        i += 1
        counters.append(0)
    
    convergence = False
    iteration = 0
    while convergence == False:
        iteration += 1
        convergence = True
        
        for idx in range(vertices_count):
            counters[idx] = 0
            
        for v in graph.vs:
            com = v["communities"][0]
            counters[com] += 1
            
        unique = sum(1 if counter > 0 else 0 for counter in counters)
        print("Iteration " + str(iteration) + ",communities " + str(unique))
        new_communities = []
        
        for v in graph.vs:
            current_community = v["communities"][0]   
            
            #clear counters
            for idx in range(vertices_count):
                counters[idx] = 0
            
            for e in v.out_edges():                    
                com = e.target_vertex["communities"][0]
                if v != e.target_vertex:
                    counters[com] += 1
                
            if sum(counters) != v.degree():
                print("Unexpected result, sum(counters) != v.degree()")
                print(str(v))
                
            max_value = max(counters)
            max_indices = []
            
            for k in range(vertices_count):
                if (counters[k] == max_value):
                    max_indices.append(k)
                    
            winner_idx = max_indices[random.randrange(len(max_indices))]
        
 
            if (current_community in max_indices):
                winner_idx = current_community
                
            if winner_idx != current_community:
                convergence = False
            
            new_communities.append([winner_idx])
        
        graph.vs["communities"] = new_communities
        #plot_full(graph, "test" + str(iteration) + ".svg", 1.5)
            
    plot_full(graph, output, 1.5)

In [24]:
#my_basic_lpa(nba11_20, "all_stars_my_1120.svg")
#my_basic_lpa(nba15_20, "all_stars_my_1520.svg")
my_basic_lpa(nba19_20, "all_stars_my_1920.svg")

Iteration 1,communities 39
Unexpected result, sum(counters) != v.degree()
igraph.Vertex(<igraph.Graph object at 0x00000290F9B1D5E8>, 5, {'name': 'Jimmy Butler', 'communities': [5], 'color': (1.0, 0.4444444444444444, 0.0, 1.0), 'label': 'Jimmy Butler\n(2)', 'size': 96})
Unexpected result, sum(counters) != v.degree()
igraph.Vertex(<igraph.Graph object at 0x00000290F9B1D5E8>, 9, {'name': 'Kevin Durant', 'communities': [9], 'color': (1.0, 0.6666666666666666, 0.0, 1.0), 'label': 'Kevin Durant\n(3)', 'size': 96})
Unexpected result, sum(counters) != v.degree()
igraph.Vertex(<igraph.Graph object at 0x00000290F9B1D5E8>, 10, {'name': 'Joel Embiid', 'communities': [10], 'color': (1.0, 0.8888888888888888, 0.0, 1.0), 'label': 'Joel Embiid\n(4)', 'size': 88})
Unexpected result, sum(counters) != v.degree()
igraph.Vertex(<igraph.Graph object at 0x00000290F9B1D5E8>, 14, {'name': 'James Harden', 'communities': [14], 'color': (0.8888888888888888, 1.0, 0.0, 1.0), 'label': 'James Harden\n(5)', 'size': 96})